In [3]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()
conn = psycopg2.connect(f"dbname={os.getenv('SQL_DATABASE')} user={os.getenv('SQL_DATABASE')} password={os.getenv('SQL_PASSWORD')} host={os.getenv('SQL_HOST')} port={os.getenv('SQL_PORT')}")
from sqlalchemy import create_engine

engine = create_engine(f"postgresql+psycopg2://{os.getenv('SQL_USER')}:{os.getenv('SQL_PASSWORD')}@{os.getenv('SQL_HOST')}:{os.getenv('SQL_PORT')}/{os.getenv('SQL_DATABASE')}")

In [8]:

# Load GeoJSON geometry
with open("fr_department_core_geozone_v2.0.json") as f:  # replace with your file path
    geojson_data = json.load(f)

gdf = gpd.GeoDataFrame.from_features(geojson_data["features"])

# Load your deployment_state data (query result)
# Example: from CSV or database
deployment_df = pd.read_sql_query(""" SELECT 
    sdc.id AS dep_geozone_id,
    CASE WHEN sds2.nb_collectivity IS NULL THEN -1
        ELSE sds2.nb_collectivity
        END AS "Nb collectivités déployées",
    CASE
        WHEN sds2.nb_collectivity IS NULL THEN 'Non planifié'
        WHEN sds2.nb_collectivity = 0 THEN 'Déploiements à venir'
        WHEN sds2.nb_collectivity > 0 AND sds2.nb_collectivity < 3 THEN 'Communes pilotes'
        WHEN sds2.nb_collectivity >= 3 AND sds2.nb_collectivity <= 5 THEN 'Généralisation en cours'
        ELSE 'Généralisation avancée'
    END AS "Etat déploiement",
    sdc.lat AS lon,
    sdc.lon AS lat
FROM analytics.stats_departments_coords sdc
LEFT JOIN analytics.stats_deployement_s2_2025 sds2 
    ON sds2.dep_geozone_id = sdc.id;""", con=conn)

# Merge geometries with deployment data
gdf = gdf.merge(deployment_df, left_on="id", right_on="dep_geozone_id")
gdf

C:\Users\Raphael\AppData\Local\Temp\ipykernel_45628\1909032059.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  deployment_df = pd.read_sql_query(""" SELECT


,geometry,id,name,name_normalized,insee_code,dep_geozone_id,Nb collectivités déployées,Etat déploiement,lon,lat
0,"MULTIPOLYGON (((3.06940 43.70677, 3.05345 43.7...",2,Hérault,herault,34,2,17,Généralisation avancée,43.579527,3.368402
1,"MULTIPOLYGON (((3.32363 44.07962, 3.29398 44.0...",360,Gard,gard,30,360,16,Généralisation avancée,43.993642,4.179888
2,"MULTIPOLYGON (((-1.03252 44.65189, -1.02556 44...",363,Gironde,gironde,33,363,9,Généralisation avancée,44.838929,-0.582924
3,"MULTIPOLYGON (((4.42165 46.20709, 4.38681 46.2...",1253,Rhône,rhone,69,1253,-1,Non planifié,45.871011,4.640838
4,"MULTIPOLYGON (((4.79539 46.22007, 4.85265 46.3...",1533,Ain,ain,01,1533,-1,Non planifié,46.099767,5.348730
...,...,...,...,...,...,...,...,...,...,...
95,"MULTIPOLYGON (((2.43548 49.13405, 2.45887 49.1...",1624,Val-d'Oise,val d oise,95,1624,-1,Non planifié,49.082740,2.130828
96,"MULTIPOLYGON (((-7.30600 50.65405, -7.31754 50...",36155,Guadeloupe,guadeloupe,971,36155,-1,Non planifié,51.000000,-7.223655
97,"MULTIPOLYGON (((-7.29531 48.94557, -7.35666 48...",36156,Martinique,martinique,972,36156,0,Déploiements à venir,49.000000,-7.223655
98,"MULTIPOLYGON (((-7.20116 45.24048, -7.09042 45...",36160,La Réunion,la reunion,974,36160,-1,Non planifié,45.000000,-7.223655


In [11]:
gdf[gdf["Nb collectivités déployées"]>-1]

,geometry,id,name,name_normalized,insee_code,dep_geozone_id,Nb collectivités déployées,Etat déploiement,lon,lat,color
0,"MULTIPOLYGON (((3.06940 43.70677, 3.05345 43.7...",2,Hérault,herault,34,2,17,Généralisation avancée,43.579527,3.368402,#3399ff
1,"MULTIPOLYGON (((3.32363 44.07962, 3.29398 44.0...",360,Gard,gard,30,360,16,Généralisation avancée,43.993642,4.179888,#3399ff
2,"MULTIPOLYGON (((-1.03252 44.65189, -1.02556 44...",363,Gironde,gironde,33,363,9,Généralisation avancée,44.838929,-0.582924,#3399ff
11,"MULTIPOLYGON (((6.90371 43.49498, 6.88404 43.5...",1540,Alpes-Maritimes,alpes maritimes,06,1540,0,Déploiements à venir,43.938066,7.116289,#ffcc99
18,"MULTIPOLYGON (((4.84156 43.33374, 4.58280 43.3...",1547,Bouches-du-Rhône,bouches du rhone,13,1547,0,Déploiements à venir,43.545654,5.085740,#ffcc99
22,"MULTIPOLYGON (((-1.41246 46.18445, -1.46715 46...",1551,Charente-Maritime,charente maritime,17,1551,0,Déploiements à venir,45.775048,-0.679041,#ffcc99
36,"MULTIPOLYGON (((-2.12504 48.59330, -2.15046 48...",1565,Ille-et-Vilaine,ille et vilaine,35,1565,1,Communes pilotes,48.155070,-1.638137,#cceeff
51,"MULTIPOLYGON (((-1.86010 49.51200, -1.88908 49...",1580,Manche,manche,50,1580,0,Déploiements à venir,49.080789,-1.328771,#ffcc99
83,"MULTIPOLYGON (((6.16020 43.00186, 6.21025 43.0...",1612,Var,var,83,1612,3,Généralisation en cours,43.441723,6.244517,#99ddff
84,"MULTIPOLYGON (((4.89536 44.33817, 4.87008 44.3...",1613,Vaucluse,vaucluse,84,1613,3,Généralisation en cours,44.007120,5.177303,#99ddff


In [ ]:
import folium
import geopandas as gpd
import json
from folium.features import DivIcon
from branca.element import MacroElement, Template
import pandas as pd
from folium import Element
# ----------------------------
# Custom color palette
# ----------------------------
state_colors = {
    'Non planifié': '#f0f0f0',          # very light gray
    'Déploiements à venir': '#ffcc99',  # light orange
    'Communes pilotes': '#cceeff',      # very light azure blue
    'Généralisation en cours': '#99ddff',  # light azure blue
    'Généralisation avancée': '#3399ff'    # medium azure blue
}

# Assign colors
gdf['color'] = gdf["Etat déploiement"].map(state_colors).fillna('#ffffff')

css = Element('<style>.leaflet-container { background-color: white; }</style>')

# ----------------------------
# Create map
# ----------------------------
m = folium.Map(
    location=[46.5, 2.5],
    zoom_start=6,
    tiles=None,
    control_scale=True,
    backGround="#ffffff"
)
m.get_root().html.add_child(css)

white_bounds = [[39, -9], [53, 12]]
# White background rectangle
folium.Rectangle(
    bounds=white_bounds,
    color="#ffffff",
    fill=True,
    fill_color="#ffffff",
    fill_opacity=1
).add_to(m)

# ----------------------------
# Add polygons and labels
# ----------------------------
for _, row in gdf.iterrows():
    sim_geo = gpd.GeoSeries(row['geometry']).simplify(0.01)
    geo_j = sim_geo.to_json()
    
    # Polygon
    folium.GeoJson(
        geo_j,
        style_function=lambda x, color=row['color']: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.85
        },
        tooltip=f"""
            <b>{row['name']}</b><br>
            Collectivités: {row['Nb collectivités déployées'] if row['Nb collectivités déployées'] is not None else 'N/A'}<br>
            État: {row['Etat déploiement'] if row['Etat déploiement'] is not None else 'N/A'}
        """
    ).add_to(m)

    # Labels (department + nb_collectivity)
    if pd.notnull(row['lat']) and pd.notnull(row['lon']):
        dep_name = row['name']
        nb_label = row.get("Nb collectivités déployées", None)
        if row['Nb collectivités déployées'] > -1:
            if row["Nb collectivités déployées"] == 0:
                label = row["name"]
            else:
                label_nb_collectivity = str(row["Nb collectivités déployées"])
                label = row["name"] + " : " + label_nb_collectivity
        else:
            label = ""  # show department name even if no deployment
            
        # Department name
        folium.map.Marker(
            [row['lon'], row['lat']],
            icon=DivIcon(
                icon_size=(150, 20),
                icon_anchor=(0, 0),
                html=f'<div class="label-name" style="text-align:left; font-weight:bold;">{label}</div>'
            )
        ).add_to(m)

# ----------------------------
# Add fixed-position legend (top-right of white rectangle)
# ----------------------------
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: absolute;
    top: 30px;                /* push down from top */
    right: 50px;              /* push from right edge */
    background-color: rgba(255,255,255,0.95);
    border: 1px solid #aaa;
    border-radius: 8px;
    padding: 10px 12px;
    width: 190px;
    box-shadow: 0 0 8px rgba(0,0,0,0.2);
    font-family: Arial, sans-serif;
    font-size: 10pt;
    z-index: 9999;
">
<b>État déploiement</b><br>
<div style="margin-top:6px; line-height: 1.4;">
    <i style="background:#f0f0f0; width:18px; height:12px; float:left; margin-right:6px; border:1px solid #999;"></i>Non planifié<br>
    <i style="background:#ffcc99; width:18px; height:12px; float:left; margin-right:6px; border:1px solid #999;"></i>Déploiements à venir<br>
    <i style="background:#cceeff; width:18px; height:12px; float:left; margin-right:6px; border:1px solid #999;"></i>Communes pilotes<br>
    <i style="background:#99ddff; width:18px; height:12px; float:left; margin-right:6px; border:1px solid #999;"></i>Généralisation en cours<br>
    <i style="background:#3399ff; width:18px; height:12px; float:left; margin-right:6px; border:1px solid #999;"></i>Généralisation avancée<br>
</div>
</div>
{% endmacro %}
"""
macro = MacroElement()
macro._template = Template(legend_html)
m.get_root().add_child(macro)

# ----------------------------
# Save to HTML
# ----------------------------
m.save("deployment_map_responsive.html")
print("✅ Map saved as 'deployment_map_responsive_v1.html'")
m

✅ Map saved as 'deployment_map_responsive.html'
